## SKOS Concept Machine Learning

In [1]:
import pandas as pd
import numpy as np
import csv
import uuid

In [2]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/skos/SKOS_ML_CLASSES.csv"

skos_ml = pd.read_csv(PATH,header=0)
display(skos_ml)

,skos:Concept,skos:inScheme,skos:broader,skos:related,rdfs:label,skos:prefLabel@fr,skos:prefLabel@en,skos:altLabel@fr,skos:altLabel@en,skos:hiddenLabel,skos:definition
0,DocumentsTypes,MLClassesList,NaN,NaN,Types de document,Types de document,NaN,NaN,NaN,NaN,Liste des types de zones contenues dans les do...
1,DoublePage,MLClassesList,DocumentsTypes,NaN,Double page,Double page,NaN,NaN,NaN,NaN,Correspondant à la numérisation d'une double p...
2,Page,MLClassesList,DocumentsTypes,NaN,Page simple,Page simple,NaN,NaN,NaN,NaN,Correspondant à la numérisation d'une page simple
3,TableHeader,MLClassesList,DocumentsTypes,NaN,En-tête de tableau,En-tête de tableau,NaN,NaN,NaN,NaN,Correspond à l'entête d'un tableau
4,TableLine,MLClassesList,DocumentsTypes,NaN,Ligne de tableau,Ligne de tableau,NaN,NaN,NaN,NaN,Correspond à une ligne de tableau
5,MATArticleDeClassement,MLClassesList,DocumentsTypes,NaN,Ligne de tableau décrivant uniquement un artic...,NaN,NaN,Ligne de tableau décrivant un état de parcelle,NaN,NaN,Compte foncier au sens document du terme : gro...
6,MATArticleDeMutation,MLClassesList,DocumentsTypes,NaN,Zone de page correspondant à un article de mut...,NaN,NaN,NaN,NaN,NaN,NaN
7,MATFolioArea,MLClassesList,DocumentsTypes,NaN,Zone qui est associée à un numéro de folio/art...,NaN,NaN,NaN,NaN,NaN,NaN
8,PageClasses,MLClassesList,NaN,NaN,Types de pages,Types de pages,NaN,NaN,NaN,NaN,Liste des classes pouvant être attribuées à un...
9,ETSTabP1,MLClassesList,PageClasses,Page,Tableau principal d'état de section (première ...,Tableau principal d'état de section (première ...,NaN,NaN,NaN,ets_tab_p1,Première page (avant 1822)/ Page unique du tab...


In [3]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastre/mlClasse/")
g.bind('mlclasse', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
g.bind('cad', cad)

# Iterate over each row in the DataFrame
for index, row in skos_ml.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, cad.MLClasse))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if row["skos:altLabel@fr"] == row["skos:altLabel@fr"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if row["skos:prefLabel@en"] == row["skos:prefLabel@en"]:
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if row["skos:altLabel@en"] == row["skos:altLabel@en"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if row["skos:broader"] == row["skos:broader"]:
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if row["skos:related"] == row["skos:related"]:
        g.add((subject_uri, SKOS.related, URIRef(baseuri + f"{str(row['skos:related'])}")))

    if row["skos:definition"] == row["skos:definition"]:
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition"],lang='fr')))

    if not pd.isnull(row['skos:hiddenLabel']):
        if ',' in row["skos:hiddenLabel"]:
            for hiddenLabel in row["skos:hiddenLabel"].split(','):
                g.add((subject_uri, SKOS.hiddenLabel, Literal(hiddenLabel)))
        else:
            g.add((subject_uri, SKOS.hiddenLabel, Literal(row["skos:hiddenLabel"])))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix mlclasse: <http://data.ign.fr/id/codes/cadastre/mlClasse/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

mlclasse:CrossedOut a cad:MLClasse,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader mlclasse:TextStyles ;
    skos:hiddenLabel "~~~~" ;
    skos:inScheme cad:MLClassesList ;
    skos:prefLabel "Texte barré/rayé"@fr .

mlclasse:DoublePage a cad:MLClasse,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader mlclasse:DocumentsTypes ;
    skos:definition "Correspondant à la numérisation d'une double page "@fr ;
    skos:inScheme cad:MLClassesList ;
    skos:prefLabel "Double page"@fr .

mlclasse:ETSBlankTab a cad:MLClasse,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader mlclasse:PageClasses ;
    skos:definition "Classe additionnelle. Désigne une page blanche (ave